In [1]:
import matplotlib.pyplot as plt
import numpy as np
dataset = np.loadtxt("history.csv", dtype=np.float32, delimiter = ",", skiprows=1)
dataset = dataset[:, 2:5]
xyz = dataset.reshape(306, 250, 3)
shape_ = xyz.shape
print(shape_)
r = []
for i in range(shape_[0]):
    for j1 in range(shape_[1]):
        for j2 in range(j1+1, shape_[1]):
            x = (xyz[i, j2, 0]-xyz[i, j1, 0])
            y = (xyz[i, j2, 1]-xyz[i, j1, 1])
            z = (xyz[i, j2, 2]-xyz[i, j1, 2])
            dr = np.sqrt(x**2+y**2+z**2)
            r.append(dr.item())

ValueError: cannot reshape array of size 232500 into shape (306,250,3)

In [ ]:
dr = 0.5
gr = []
nx = int(float(max(r))/dr)
rl = [dr*i for i in range(1, nx)]
for drl in rl:
    count = 0
    for item in r:
        if item > drl and item <= drl+dr:
            count += 1
    gr.append(count/(4*3.14*drl*dr))

In [ ]:
from matplotlib import pyplot as plt
plt.plot(rl, gr)
plt.ylabel('RDF')
plt.xlabel('r')
#plt.xlim(0, 30)